## Импорты

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchaudio

import yt_dlp
import subprocess

import os
import pickle
import gc
import json
import locale
import re
import tqdm.notebook as tqdm
from urllib.parse import urlparse
import requests
import math

import av
from huggingface_hub import hf_hub_download

from transformers import VivitImageProcessor, VivitModel
from transformers import AutoImageProcessor, VideoMAEModel
from transformers import TimesformerConfig, TimesformerModel
from transformers import XCLIPProcessor, XCLIPModel
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

import librosa
from moviepy.editor import VideoFileClip
import ast
import openunmix

from minio import Minio
from minio.error import S3Error

import hydra
import soundfile as sf
from omegaconf import OmegaConf

import logging
from typing import List

import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

locale.getpreferredencoding = lambda: "UTF-8"

logging.basicConfig(
    filename='vivit_inference.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

Константы

In [ ]:
CATEGORIES_ENG = """Auto-moto,
Anime,
Audiobooks,
Business,
Video games,
Interview,
Art,
Movie,
Beauty,
Cooking,
Life Hacks,
Music,
Cartoons,
News,
Training,
Hunting and fishing,
Politics,
Psychology,
Journeys,
Serials,
Sport,
Humor,
Lifestyle,
Realty,
Health,
Nature,
Design,
Machinery and equipment,
Business and entrepreneurship,
Culture,
Religion,
Construction and renovation,
Garden and vegetable garden,
Food,
Entertainment,
Esotericism,
The science,
Audio,
Technology and the Internet,
TV shows,
For children,
Hobby,
Various,
Animals,
News and media,
Films,
Bloggers,
"""

CATEGORIES_RUS = """
Авто-мото
Аниме
Аудиокниги
Бизнес
Видеоигры
Интервью
Искусство
Кино
Красота
Кулинария
Лайфхаки
Музыка
Мультфильмы
Новости
Обучение
Охота_и_рыбалка
Политика
Психология
Путешествия
Сериалы
Спорт
Юмор
Лайфстайл
Недвижимость
Здоровье
Природа
Дизайн
Техника_и_оборудование
Бизнес_и_предпринимательство
Культура
Религия
Строительство_и_ремонт
Сад_и_огород
Еда
Развлечения
Эзотерика
Наука
Аудио
Технологии_и_интернет
Телепередачи
Детям
Хобби
Разное
Животные
Новости_и_СМИ
Фильмы
Блогеры
"""


RUS_TEXT_PROMPT = [
    f"Видео принадлежит категории '{x}'" for x in CATEGORIES_RUS.split() if x.strip() != ''
]
ENG_TEXT_PROMPT = [
    f"Video belongs to category '{x}'" for x in CATEGORIES_ENG.split(',\n') if x.strip() != ''
]

## Download video

In [ ]:
client = Minio(
    endpoint="storage.yandexcloud.net",
    access_key="YCAJESQqZUja9X-F1glArEPSY",
    secret_key="YCP6M_QUdKUF1XBlgz_hOWAlTkcMbnEUyLG5hsQv",
)

BUCKET_NAME = "rutube-tagging"


def list_files_in_bucket(bucket_name=BUCKET_NAME):
    """
    List all files in the S3 bucket.
    """
    try:
        objects = client.list_objects(bucket_name)
        for obj in objects:
            print(obj.object_name)
    except S3Error as e:
        print(f"Error listing objects in bucket: {e}")


def download_video_from_s3_url(url, output_directory='.'):
    """
    Downloads a video from an S3-compatible URL using the MinIO client.

    :param url: The S3 URL of the video to download.
    :param output_directory: The directory where the video will be saved.
    """
    try:
        # Parse the URL to extract bucket and object name
        parsed_url = urlparse(url)
        path_parts = parsed_url.path.lstrip('/').split('/', 1)

        if len(path_parts) != 2:
            raise ValueError("URL does not contain both bucket and object name.")

        bucket_name, object_name = path_parts

        # Ensure the output directory exists
        os.makedirs(output_directory, exist_ok=True)

        # Define the local file path
        filename = os.path.basename(object_name)
        local_file_path = os.path.join(output_directory, filename)

        print(f"Starting download of {url} using MinIO client...")

        # Download the object
        client.fget_object(
            bucket_name, object_name, local_file_path
        )

        print(f"Download completed. Saved to {local_file_path}")

    except S3Error as e:
        print(f"S3 error occurred: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


def download_video_from_url(url, output_directory='.'):
    """
    Downloads a video from the given URL to the specified output directory.

    :param url: The URL of the video to download.
    :param output_directory: The directory where the video will be saved.
    """
    try:
        # Ensure the output directory exists
        os.makedirs(output_directory, exist_ok=True)

        # Extract the filename from the URL
        filename = os.path.basename(url)
        local_file_path = os.path.join(output_directory, filename)

        print(f"Starting download of {url}...")

        # Stream the download to handle large files
        with requests.get(url, stream=True) as response:
            response.raise_for_status()  # Check for HTTP errors
            with open(local_file_path, 'wb') as f:
                for chunk in tqdm.tqdm(response.iter_content(chunk_size=8192)):
                    if chunk:  # Filter out keep-alive chunks
                        f.write(chunk)

        print(f"Download completed. Saved to {local_file_path}")

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred while downloading {url}: {http_err}")
    except Exception as err:
        print(f"An error occurred while downloading {url}: {err}")

In [ ]:
np.random.seed(0)
torch.manual_seed(0)


def get_model_params_count(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def sample_frame_indices(desired_frames: int, frame_sample_rate: int, total_frames: int) -> np.ndarray:
    indices = np.arange(0, frame_sample_rate*desired_frames, frame_sample_rate)
    if indices[-1] >= total_frames:
        indices = np.clip(indices, 0, total_frames-1)

    return indices


def read_video_pyav(container: av.container.input.InputContainer, indices: List[int]) -> np.ndarray:
    from collections import Counter

    frames = []
    container.seek(0)
    indices_dct = Counter(indices)

    max_idx = indices[-1]

    for i, frame in enumerate(container.decode(video=0)):
        if i > max_idx:
            break

        if i in indices_dct:
            while indices_dct[i] > 0:
                frames.append(frame)
                indices_dct[i] -= 1

    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def process_video(
    video_path: str,
    image_processor,
    model,
    device: str = 'cuda',
    desired_frames: int = 96,
    window_size: int = 32,
    xclip_text_prompt: list = RUS_TEXT_PROMPT
) -> torch.Tensor:
    """
    Processes a Rutube video: extracts the first two minutes, samples 96 frames,
    and returns the ViViT model's output.

    :param video_path: Path to the video file.
    :param window_size: Size of each window for ViViT.
    :return: Model's output tensor.
    """
    assert desired_frames % window_size == 0

    try:
        # Open video with PyAV
        container = av.open(video_path)
        video_stream = container.streams.video[0]

        # Get total frames
        total_frames = video_stream.frames
        if total_frames == 0:
            duration = float(video_stream.duration * video_stream.time_base)
            fps = float(video_stream.average_rate)
            total_frames = int(duration * fps)
        else:
            fps = float(video_stream.average_rate)
            duration = float(video_stream.duration * video_stream.time_base)

        # Determine target duration (first minute or actual duration)
        target_duration = min(60, duration)  # seconds

        # Calculate frame_sample_rate and frame indices, read sampled frames
        frame_sample_rate = max(1, int(math.floor((fps * target_duration) / desired_frames)))
        indices = sample_frame_indices(desired_frames, frame_sample_rate, total_frames)
        video_frames = read_video_pyav(container, indices)

        if len(video_frames) == 0:
            raise ValueError("No frames were extracted from the video.")

        # Process inputs with image_processor
        if isinstance(model, XCLIPModel):
            inputs = image_processor(
                text=xclip_text_prompt,
                videos=list(video_frames),
                return_tensors="pt",
                padding=True,
                max_length=1024
            )
        else:
            inputs = image_processor(
                list(video_frames),
                return_tensors="pt",
                padding=True
            )
        inputs['pixel_values'] = torch.concat(torch.split(inputs['pixel_values'], window_size, dim=1))
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Forward pass
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract desired output
        if hasattr(outputs, 'pooler_output'):
            output = outputs.pooler_output.mean(dim=0).cpu()
            return output
        elif hasattr(outputs, 'last_hidden_state'):
            output = outputs.last_hidden_state.mean(dim=[0, 1]).cpu()
            return output
        elif isinstance(model, XCLIPModel):
            probs = outputs.logits_per_video.cpu().squeeze().softmax(dim=-1)
            video_emb = outputs.video_embeds.cpu().squeeze()
            return video_emb, probs
        else:
            raise AttributeError("Model output does not contain 'pooler_output' or 'last_hidden_state'.")

        return output

    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        print("An unexpected error occurred:")
        print(e)


def instantiate_model(model_name: str, device: str = 'cuda'):
    if model_name == 'vivit':
        model = VivitModel.from_pretrained("models/vivit_model")
        image_processor = VivitImageProcessor.from_pretrained("models/vivit_image_processor")
        window_size = 32
    elif model_name == 'timesformer':
        model = TimesformerModel.from_pretrained("models/timesformer_model")
        image_processor = AutoImageProcessor.from_pretrained("models/timesformer_image_processor")
        window_size = 8
    elif model_name == 'videomae':
        model = VideoMAEModel.from_pretrained("models/videomae_model")
        image_processor = AutoImageProcessor.from_pretrained("models/videomae_image_processor")
        window_size = 16
    elif model_name == 'x-clip':
        model = XCLIPModel.from_pretrained("models/xclip_model")
        image_processor = XCLIPProcessor.from_pretrained("models/xclip_image_processor")
        window_size = 32
    else:
        raise ValueError(f"{model_name} is not supported, should be in ('vivit', 'timesformer', 'videomae', 'x-clip')")


    model = model.to(device)
    model.eval()

    return model, image_processor, window_size

## Calculate embeddings

### Save storage

In [ ]:
class EmbeddingStorage:
    def __init__(self, labels=None, filenames=None, embeddings=None):
        """
        Initialize the EmbeddingStorage class.

        Args:
            labels (list or np.ndarray): An array of labels for the embeddings.
            filenames (list or np.ndarray): An array of filenames associated with the embeddings.
            embeddings (np.ndarray): A NumPy array containing all embeddings.
        """
        self.labels = np.array(labels) if labels is not None else np.array([])
        self.filenames = np.array(filenames) if filenames is not None else np.array([])
        self.embeddings = np.array(embeddings) if embeddings is not None else np.empty((0,))

    def add_embedding(self, label, filename, embedding):
        """
        Add a new embedding, along with its label and filename.

        Args:
            label (int or str): The label of the embedding.
            filename (str): The filename associated with the embedding.
            embedding (np.ndarray or torch.Tensor): The embedding to add (can be a NumPy array or Tensor).
        """
        if isinstance(embedding, np.ndarray):
            emb_array = embedding
        else:
            # Convert torch.Tensor to NumPy
            emb_array = embedding.cpu().numpy()

        # Append the new data
        self.labels = np.append(self.labels, label)
        self.filenames = np.append(self.filenames, filename)

        if self.embeddings.size == 0:
            self.embeddings = emb_array.reshape(1, -1)
        else:
            self.embeddings = np.vstack([self.embeddings, emb_array])

    def save_to_file(self, file_path):
        """
        Save the embeddings, labels, and filenames to a file (as .npz).

        Args:
            file_path (str): The path to save the .npz file.
        """
        np.savez(file_path, labels=self.labels, filenames=self.filenames, embeddings=self.embeddings)

    @classmethod
    def load_from_file(cls, file_path):
        """
        Load embeddings, labels, and filenames from a saved .npz file.

        Args:
            file_path (str): The path to the .npz file to load.

        Returns:
            EmbeddingStorage: An instance of EmbeddingStorage with loaded data.
        """
        data = np.load(file_path)
        return cls(labels=data['labels'], filenames=data['filenames'], embeddings=data['embeddings'])

    def get_embedding_by_filename(self, filename):
        """
        Retrieve an embedding by its associated filename.

        Args:
            filename (str): The filename to search for.

        Returns:
            np.ndarray: The corresponding embedding or None if not found.
        """
        if filename in self.filenames:
            idx = np.where(self.filenames == filename)[0][0]
            return self.embeddings[idx]
        else:
            return None

    def __len__(self):
        """
        Return the number of embeddings stored.
        """
        return len(self.labels)

    def __getitem__(self, idx):
        """
        Retrieve the label, filename, and embedding by index.

        Args:
            idx (int): The index of the embedding to retrieve.

        Returns:
            tuple: A tuple containing (label, filename, embedding).
        """
        if idx >= len(self.labels):
            raise IndexError("Index out of range")
        return self.labels[idx], self.filenames[idx], self.embeddings[idx]

    def __repr__(self):
        return f"EmbeddingStorage(labels={len(self.labels)}, filenames={len(self.filenames)}, embeddings_shape={self.embeddings.shape})"

### Video

In [ ]:
if 'mapping_df' in locals():
    del mapping_df

mapping_df = pd.read_csv('train_data_categories.csv')

#### Upload rest of the videos

In [ ]:
import os


data_dir = './data/'
directory = os.fsencode(data_dir)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    fullpath = os.path.join(data_dir, filename)

    if os.path.isdir(fullpath):
        if len(os.listdir(fullpath)) <= 6:
            print(filename, len(os.listdir(fullpath)))

In [ ]:
def extract_s3_id(s3_url: str) -> str:
    if s3_url is np.nan:
        return ''

    last_backslash_idx = len(s3_url) - s3_url[::-1].find('/')
    return s3_url[last_backslash_idx:]


def remove_slashes(s: str) -> str:
    if not isinstance(s, str):
        return ''

    chars_to_remove = [
        '/', '\\',            # Standard slashes
        "'", '"',             # Standard quotes
        '«', '»',             # Guillemets
        '“', '”',             # Curly double quotes
        '‘', '’',             # Curly single quotes
        '＂',                 # Fullwidth double quote
        '⧸',                   # Slash-like character
        '｜', '│',            # Vertical bar-like characters
        # Add any other auxiliary characters as needed
    ]

    translation_table = str.maketrans('', '', ''.join(chars_to_remove))
    cleaned_string = s.translate(translation_table)
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string).strip()

    return cleaned_string


CATEGORIES_TO_UPLOAD = [
    "48.0_films",
    "19.0_serials",
    "7.0_movie",
    "29.0_design",
    "46.0_animals",
    "45.0_various",
    "5.0_interview",
    "42.0_tv shows",
    "35.0_garden and vegetable garden",
]

mapping_df = pd.read_csv('scraped_dataset.csv')
mapping_df['s3_name'] = mapping_df['s3_url'].apply(extract_s3_id)
mapping_df['video_name'] = mapping_df['parsed_category'].apply(lambda tuple_str: ast.literal_eval(tuple_str)[0])
mapping_df.dropna(subset=['video_name'], inplace=True)

data_dir = './data/'

for cat in tqdm.tqdm(CATEGORIES_TO_UPLOAD):
    ctg_id = extract_category_id(cat)
    df_slice = mapping_df.query(f"manual_category_id == {ctg_id}")

    subdir_name = os.path.join(data_dir, cat)
    downloaded_videos = list(map(remove_slashes, os.listdir(subdir_name)))

    for id, row in df_slice.iterrows():
        video_name = remove_slashes(row['video_name'])

        if video_name + '.mp4' not in downloaded_videos:
            download_video_from_url(row['s3_url'], subdir_name)
            # print(row['url'], subdir_name)
            # download_rutube_video_first_two_minutes(row['url'], subdir_name)


#### Inference ViViT

In [ ]:
def extract_category_id(ctgry: str) -> int:
    return int(float(ctgry[:3]))


DATA_DIR = './data'
model_name = 'vivit'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, processor, window_size = instantiate_model(model_name, device)
embeddings_vivit = []

for data_subdir in tqdm.tqdm(os.listdir(DATA_DIR)):
    fullpath = os.path.join(DATA_DIR, data_subdir)

    if not os.path.isdir(fullpath):
        continue

    video_files = [f for f in os.listdir(fullpath) if f.lower().endswith('.mp4')]
    logging.info(f"Found {len(video_files)} video(s) in directory '{data_subdir}'.")

    for video_file in video_files:
        file_path = os.path.join(fullpath, video_file)
        output = process_video(file_path, processor, model, device, desired_frames=96, window_size=window_size)

        if output is not None:
            logging.info(f"Successfully processed video: {video_file}")
            embeddings_vivit.append((video_file, extract_category_id(data_subdir), output.numpy()))
        else:
            logging.warning(f"Failed to process video: {video_file}")

vivit_df = EmbeddingStorage(
    filenames=list(map(lambda x: x[0], embeddings_vivit)),
    labels=list(map(lambda x: x[1], embeddings_vivit)),
    embeddings=list(map(lambda x: x[2], embeddings_vivit)),
)
vivit_df.save_to_file('data/embeddings/vivit.npz')

# vivit_df = pd.DataFrame(data=embeddings_vivit, columns=['videoname', 'label', 'emb'])
# vivit_df.to_csv('data/embeddings/vivit.csv', index=False)

  0%|          | 0/48 [00:00<?, ?it/s]

#### Inference X-CLIP

In [ ]:
del model, processor
torch.cuda.empty_cache()

In [ ]:
def extract_category_id(ctgry: str) -> int:
    return int(float(ctgry[:3]))


DATA_DIR = './data'
model_name = 'x-clip'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, processor, window_size = instantiate_model(model_name, device)
embeddings_xclip = []

for data_subdir in tqdm.tqdm(os.listdir(DATA_DIR)):
    fullpath = os.path.join(DATA_DIR, data_subdir)

    if not os.path.isdir(fullpath):
        continue

    video_files = [f for f in os.listdir(fullpath) if f.lower().endswith('.mp4')]
    logging.info(f"Found {len(video_files)} video(s) in directory '{data_subdir}'.")

    for video_file in video_files:
        file_path = os.path.join(fullpath, video_file)
        output = process_video(file_path, processor, model, device, desired_frames=96, window_size=window_size)

        if output is not None:
            logging.info(f"Successfully processed video: {video_file}")
            embeddings_xclip.append((video_file, extract_category_id(data_subdir), output[0].numpy(), output[1].numpy()))
        else:
            logging.warning(f"Failed to process video: {video_file}")

xclip_emb_df = EmbeddingStorage(
    filenames=list(map(lambda x: x[0], embeddings_xclip)),
    labels=list(map(lambda x: x[1], embeddings_xclip)),
    embeddings=list(map(lambda x: x[2], embeddings_xclip)),
)
xclip_emb_df.save_to_file('data/embeddings/xclip_emb.npz')

xclip_prob_df = EmbeddingStorage(
    filenames=list(map(lambda x: x[0], embeddings_xclip)),
    labels=list(map(lambda x: x[1], embeddings_xclip)),
    embeddings=list(map(lambda x: x[3], embeddings_xclip)),
)
xclip_prob_df.save_to_file('data/embeddings/xclip_prob.npz')

# xclip_df = pd.DataFrame(data=embeddings_xclip, columns=['videoname', 'label', 'emb', 'prob'])
# xclip_df.to_csv('data/embeddings/xclip.csv', index=False)

  0%|          | 0/48 [00:00<?, ?it/s]

#### Inference Video-MAE

In [ ]:
del model, processor
torch.cuda.empty_cache()

In [ ]:
def extract_category_id(ctgry: str) -> int:
    return int(float(ctgry[:3]))


DATA_DIR = './data'
model_name = 'videomae'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, processor, window_size = instantiate_model(model_name, device)
embeddings_videomae = []

for data_subdir in tqdm.tqdm(os.listdir(DATA_DIR)):
    fullpath = os.path.join(DATA_DIR, data_subdir)

    if not os.path.isdir(fullpath):
        continue

    video_files = [f for f in os.listdir(fullpath) if f.lower().endswith('.mp4')]
    logging.info(f"Found {len(video_files)} video(s) in directory '{data_subdir}'.")

    for video_file in video_files:
        file_path = os.path.join(fullpath, video_file)
        output = process_video(file_path, processor, model, device, desired_frames=96, window_size=window_size)

        if output is not None:
            logging.info(f"Successfully processed video: {video_file}")
            embeddings_videomae.append((video_file, extract_category_id(data_subdir), output.numpy()))
        else:
            logging.warning(f"Failed to process video: {video_file}")

videomae_df = EmbeddingStorage(
    filenames=list(map(lambda x: x[0], embeddings_videomae)),
    labels=list(map(lambda x: x[1], embeddings_videomae)),
    embeddings=list(map(lambda x: x[2], embeddings_videomae)),
)
videomae_df.save_to_file('data/embeddings/videomae.npz')

# videomae_df = pd.DataFrame(data=embeddings_videomae, columns=['videoname', 'label', 'emb'])
# videomae_df.to_csv('data/embeddings/videonae.csv', index=False)

  0%|          | 0/48 [00:00<?, ?it/s]

## X-CLIP

### No context

In [ ]:
tag_id_to_index

In [ ]:
clip_df = pd.read_csv('data/embeddings/x-clip-v2.csv')
tag_id_to_index = dict(zip(label_df.tag_id, label_df.index))

Accuracy of first window

In [ ]:
clip_id_label = clip_df.label.apply(lambda x: tag_id_to_index[x]).values
top_n_predictions = np.vstack(clip_df.probs.apply(lambda x: np.argsort(string_to_numpy(x))[0, :][::-1]))

print('Top 1 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 1) * 100)
print('Top 5 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 5) * 100)
print('Top 10 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 10) * 100)

Top 1 accuracy: 8.16831683168317
Top 5 accuracy: 19.554455445544555
Top 10 accuracy: 32.17821782178218


Accuracy of second window

In [ ]:
clip_id_label = clip_df.label.apply(lambda x: tag_id_to_index[x]).values
top_n_predictions = np.vstack(clip_df.probs.apply(lambda x: np.argsort(string_to_numpy(x))[1, :][::-1]))

print('Top 1 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 1) * 100)
print('Top 5 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 5) * 100)
print('Top 10 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 10) * 100)

Top 1 accuracy: 9.158415841584159
Top 5 accuracy: 20.049504950495052
Top 10 accuracy: 31.18811881188119


Accuracy of third window

In [ ]:
clip_id_label = clip_df.label.apply(lambda x: tag_id_to_index[x]).values
top_n_predictions = np.vstack(clip_df.probs.apply(lambda x: np.argsort(string_to_numpy(x))[2, :][::-1]))

print('Top 1 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 1) * 100)
print('Top 5 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 5) * 100)
print('Top 10 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 10) * 100)

Top 1 accuracy: 5.445544554455446
Top 5 accuracy: 20.049504950495052
Top 10 accuracy: 30.198019801980198
